In [1]:
import logging
import json
import torch
from types import SimpleNamespace
from vilbert.vilbert import VILBertActionSelection, BertConfig
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule
import torch.distributed as dist
from VLN_config import config as args
from torch.nn import CrossEntropyLoss



In [2]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [3]:
# Input:
# - instruction.
# - image.

# train_dataset =
# val_dataset =


In [4]:
config = BertConfig.from_json_file(args.config_file)
bert_weight_name = json.load(
    open("config/" + args.bert_model + "_weight_name.json", "r")
)

tokenizer = BertTokenizer.from_pretrained(
    args.bert_model, do_lower_case=args.do_lower_case
)

config.track_temporal_features = args.track_temporal_features
config.mean_layer = args.mean_layer
config.max_temporal_memory_buffer = args.max_temporal_memory_buffer


model = VILBertActionSelection.from_pretrained(
    args.from_pretrained, config=config, default_gpu=True
)

07/15/2020 13:50:57 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/salem/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
07/15/2020 13:50:57 - INFO - vilbert.utils -   loading weights file save/multitask_model/multi_task_model.bin
07/15/2020 13:51:02 - INFO - vilbert.utils -   Weights of VILBertActionSelection not initialized from pretrained model: ['positional_enc.weight', 'positional_enc.bias', 'img_emb_mean.weight', 'img_emb_mean.bias', 'vil_prediction_action_selection.logit_fc.0.weight', 'vil_prediction_action_selection.logit_fc.0.bias', 'vil_prediction_action_selection.logit_fc.2.weight', 'vil_prediction_action_selection.logit_fc.2.bias', 'vil_prediction_action_selection.logit_fc.3.weight', 'vil_prediction_action_selection.logit_fc.3.bias']
07/15/202

In [5]:
model

VILBertActionSelection(
  (positional_enc): Linear(in_features=7, out_features=2048, bias=True)
  (img_emb_mean): Linear(in_features=10240, out_features=2048, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (v_embeddings): BertImageEmbeddings(
      (image_embeddings): Linear(in_features=2048, out_features=1024, bias=True)
      (image_location_embeddings): Linear(in_features=5, out_features=1024, bias=True)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True

In [5]:
for key, value in dict(model.named_parameters()).items():
    if not value.requires_grad:
        print("This parameter does have grad", key)

In [ ]:
criterion = CrossEntropyLoss()